In [1]:
#-------------------------------------------------------------------------------
# Name:        module2
# Purpose:
#
# Author:      beckymcfarland
#
# Created:     07/11/2024
# Copyright:   (c) beckymcfarland 2024
# Licence:     <your licence>
#-------------------------------------------------------------------------------

def main():
    pass

if __name__ == '__main__':
    main()




import aiohttp
import asyncio
import io
import requests
import csv
import os
import json
import sys
import pandas as pd
import time
import subprocess
import calendar
from typing import List, Dict
from pandas import json_normalize
from datetime import date
from datetime import timedelta
from datetime import datetime, timezone
from datetime import datetime as dt
import win32com.client
import plotly
import dateutil

import chart_studio.plotly as py


import nest_asyncio
nest_asyncio.apply()

#from plotly import offline
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pyo.init_notebook_mode(connected=True)


pd.options.plotting.backend = 'plotly'


#**************************TO BE RUN ON JUPYTER NOTEBOOK********************************


class Client:
    def __init__(self, client_id: str, client_secret: str, refresh_token: str, expiration_buffer: int = 30) -> None:
        self.client_id = client_id
        self.client_secret = client_secret
        self.refresh_token = refresh_token
        self.access_token = ""
        self.expiration_buffer = expiration_buffer
        self.expires_in = 0
        self.time_of_last_refresh = 0
        self.auth_url = "https://signin.tradestation.com/oauth/token"

    async def get_access_token(self, force: bool = False) -> str:
        seconds_since_last_token = time.time() - self.time_of_last_refresh

        if force or seconds_since_last_token > self.expires_in - self.expiration_buffer:
            payload = {
                "grant_type": "refresh_token",
                "client_id": self.client_id,
                "client_secret": self.client_secret,
                "refresh_token": self.refresh_token,
            }
            async with aiohttp.ClientSession() as session:
                async with session.post(self.auth_url, data=payload) as response:
                    json_data = await response.json()
                    self.time_of_last_refresh = time.time()
                    self.expires_in = int(json_data["expires_in"])
                    self.access_token = json_data["access_token"]

        return self.access_token


UNIT = "Minute"
INTERVAL = 5
BARS_BACK = 100



async def fetch_and_process_data(symbol: str, client: Client, unit: str, interval: int, bars_back: int):
    while True:
        url = f"https://api.tradestation.com/v3/marketdata/barcharts/{symbol}"
        params = {
            "interval": interval,
            "unit": unit,
            "barsback": bars_back,
        }
        access_token = await client.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}

        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(url, params=params, headers=headers) as response:
                    if response.status == 200:
                        data = await response.json()
                        print(f"Raw data received for {symbol}: {data.keys()}")

                        if 'Bars' in data:
                            print(f"Number of bars received for {symbol}: {len(data['Bars'])}")
                            result = await process_bars(data['Bars'], symbol, client)
                            if result:
                                return
                            else:
                                print(f"{symbol}: Condition not met, waiting 5 minutes before retrying...")
                                await asyncio.sleep(300)
                        else:
                            print(f"No bars data found for {symbol}. Full response: {data}")
                            return

        except Exception as e:
            print(f"Error in process_bars for {symbol}: {e}")
            return False



async def process_bars(bars: List[Dict], symbol: str, client: Client) -> bool:
    try:
        # Convert bars to DataFrame
        df = pd.DataFrame(bars)
        #print(df)
        df = df.sort_values('TimeStamp', ascending=False)  # Most recent first
        ## Remove for now ## TimeStamps = df['TimeStamp'].iloc[1:].head(10).tolist()
        #print(f" Last bar TimeStamp for {symbol}: {TimeStamps[0]}")
        #print(df)


        # Verify we have data
        if df.empty:
            print(f"No data found for {symbol}")
            return False

        # Sort and get timestamps
      
        #print(f"Columns in dataframe for {symbol}: {df.columns}")
        #print(f"Number of bars received: {len(df)}")

        # Print first few timestamps to verify data
        latest_timestamps = df['TimeStamp'].head(3).tolist()

 
        def convert_time_in_list(time_list, input_format, output_format):
            time_list = latest_timestamps
            #input_format = "%Y-%m-%dT%H:%M:%S.000Z"  
            output_format = "%H:%M"
        
            converted_times = []
            for time_str in time_list:
                try:
                    #dt_object = datetime.strptime(time_str, input_format)
                    converted_time = time_str.strftime(output_format)
                    #converted_times.append(converted_time)
                except ValueError:
                    converted_times.append("Invalid format")
        print(converted_time)                    

        #return converted_times






       
        #print(f"Latest timestamps for {symbol}: {latest_timestamps}")
        #Close = df.Close 
        
        
        # If we got here and have data, return True
        #return True if len(df) > 0 else False
 
        sym_str = str(symbol)
        print(sym_str)

       # T1 = '06:00'
        
       # format = '%H:%M'
       # time = datetime.strptime(s2, format) - datetime.strptime(T1, format)


        
        #df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
        #df['date_without_year'] = df['TimeStamp'].dt.strftime('%H:%M')
    # #   date_without_year = latest_timestamps.dt.strftime('%H:%M')
    # #   print(date_without_year)

        #T1 = '06:00'
        
        #format = '%H:%M'
       # time = datetime.strptime(s2, format) - datetime.strptime(T1, format)


        #print(df['date_without_month'])
        #print(f"Columns in dataframe for {symbol}: {df.columns}")
        #print(df)

      


       
       
        df2 = df.loc[:, ('Open', 'High', 'Low', 'Close', 'TimeStamp', 'TotalVolume')]
        #df2 = (df[['Open', 'High', 'Low', 'Close', 'TimeStamp', 'TotalVolume']])
        df2.drop('TimeStamp', axis=1, inplace=True)
        
        
        #print(df2)
       
        #df2.pop('TimeStamp')
        df2.columns =['Open', 'High', 'Low', 'Close', 'TotalVolume']
        df2 = df2.astype(float)
        df2 = df2.head(10)
        
        fig = make_subplots(rows=2, cols=1, 
                            shared_xaxes=True, 
                            vertical_spacing=0.10, 
                            subplot_titles=('Price', 'Volume'), 
                            row_width=[0.3, 0.7])   

        
        # Add candlestick to top subplot only
        fig.add_trace(go.Candlestick(x=df2.index, 
                                    open=df2["Open"], 
                                    high=df2["High"],
                                    low=df2["Low"], 
                                    close=df2["Close"], 
                                    name="OHLC"),
                      row=1, col=1)

        
        df2['MA10'] = df2['Close'].rolling(window=10, min_periods=0).mean()

        
        # Add MA10 to top subplot only
        fig.add_trace(go.Scatter(x=df2.index, 
                                y=df2["MA10"], 
                                marker_color='grey',
                                name="MA10"), 
                      row=1, col=1)

        # Add volume to bottom subplot only
        fig.add_trace(go.Bar(x=df2.index, 
                             y=df2['TotalVolume'], 
                             marker_color='red', 
                             showlegend=False), 
                      row=2, col=1)
        
        # Update layout with separate y-axis titles
        fig.update_layout(
            title=f'{symbol} Historical Price Chart',
            yaxis_title='Price ($/share)',
            yaxis2_title='Volume',
            xaxis_tickfont_size=12,
            autosize=False,
            width=800,
            height=500,
            margin=dict(l=50, r=50, b=100, t=100, pad=4),
            paper_bgcolor='LightSteelBlue'
        )
        
        # Update y-axes separately
        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Volume", row=2, col=1)
        
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show(renderer="colab")

     


       # # # # # If we got here and have data, return True
        return True if len(df2) > 0 else False

    except Exception as e:
            print(f"Error in process_bars for {symbol}: {e}")
            return False


async def main():

    cols = ['SYMBOL']
    df = pd.read_csv('c:\\Users\\BeckyMcFarland\\AppData\\Local\\Programs\\Python\\Python312\\SCPSymbols.csv',
                    names=cols)


    client = Client(
        client_id="9yqHGFAuqBulr0Fdw6z0bp6Cj4jD1Zzb",
        client_secret="Pnduh5O1PORKocWafm6P5ZR1-sw6DS4PDSPwkgZYI4IBxYGBREoMlto7r_FVBVkq",
        refresh_token="Nr8a_hfH_q2xzp62o8Ux1UYwFCsYra6DyHlaxWEA-9o3e"
    )

    tasks = []
    for _, row in df.iterrows():
        symbol = row['SYMBOL']
        task = asyncio.create_task(fetch_and_process_data(symbol, client, UNIT, INTERVAL, BARS_BACK))
        tasks.append(task)

    await asyncio.gather(*tasks)



if __name__ == "__main__":
    asyncio.run(main())





Raw data received for AAPL: dict_keys(['Bars'])
Number of bars received for AAPL: 100
Error in process_bars for AAPL: name 'converted_time' is not defined
AAPL: Condition not met, waiting 5 minutes before retrying...
Raw data received for NVDA: dict_keys(['Bars'])
Number of bars received for NVDA: 100
Error in process_bars for NVDA: name 'converted_time' is not defined
NVDA: Condition not met, waiting 5 minutes before retrying...
Raw data received for META: dict_keys(['Bars'])
Number of bars received for META: 100
Error in process_bars for META: name 'converted_time' is not defined
META: Condition not met, waiting 5 minutes before retrying...
Raw data received for V: dict_keys(['Bars'])
Number of bars received for V: 100
Error in process_bars for V: name 'converted_time' is not defined
V: Condition not met, waiting 5 minutes before retrying...
Raw data received for JPM: dict_keys(['Bars'])
Number of bars received for JPM: 100
Error in process_bars for JPM: name 'converted_time' is not 

RuntimeError: Event loop stopped before Future completed.